### Hamiltonian convention

The k-dependent Hamiltonian is read from files in

```
N_KPOINTS N_WANNIER N_IGNORED
kx1 ky1 kz1
H_sp0_orb0_sp0_orb0 H_sp0_orb0_sp0_orb1 ...
H_sp0_orb1_sp0_orb0 H_sp0_orb1_sp0_orb1 ...
...
kx2 ky2 kz2
...
```

format with optional spin-dependence (specified in config). Rows/columns are never transposed on the way to Hloc, but orb/sp in each direction are reordered to agree with usual w2dynamics `orb,sp` ordering convention. Hloc is the average over kpoints of the read-in Hamiltonian file, and passed as 1p local Hamiltonian to QMC is (strictly speaking the negation of) `muimp = mu * eye - dc_full - Re(Hloc) - sigma_hartree`.

The elements $H_{ij}$ of minus muimp, i.e. QMC muimp, containing a term proportional to the elements of Hloc, are used in QMC as quadratic part of the Hamiltonian
$$
H = \sum_{i, j} c^{\dagger}_i H_{ij} c_j.
$$

### U-matrix convention

The local interaction matrix is read from files in 

```
N_BANDS BANDS
i1 j1 k1 l1 Re(U_i1j1k1l1)
i2 j2 k2 l2 Re(U_i2j2k2l2)
...
```

format, for spin-dependent interaction with d/u appended to each index to indicate spin-index 0/1. It is passed to QMC directly, reshaped to 4 flavor dimensions. The elements $U_{ijkl}$ are used in QMC as quartic part of the Hamiltonian
$$
H = \frac{1}{2}\sum_{i,j,k,l} c^{\dagger}_i c^{\dagger}_j U_{ijkl} c_l c_k.
$$

#### Matrix element index conventions

In our formula for H, the matrix element in terms of the operator as a function of integration variables (in real-space representation) is, cf Schwabl QM2, 
$$
U_{ijkl} = \int \mathrm{d}x \int \mathrm{d}y \phi_i^{*}(x) \phi_j^{*}(y) U(x, y) \phi_k(x) \phi_l(y).
$$

The alternative chemists' notation is
$$
[ij|kl] = \int \mathrm{d}x \int \mathrm{d}y \phi_i^{*}(x) \phi_j(x) U(x, y) \phi_k^{*}(y) \phi_l(y),
$$
and so $U_{ijkl} = [ik|jl]$.

### Rotation convention

The rotation matrix is read as simple two-dimensional matrix with imaginary part in every second column from a text file.

The rotation is applied to the quadratic part of the Hamiltonian as $H'_{ij} = (R^{\dagger})_{ik}H_{kl}R_{lj}$, and it fulfills $(R^{\dagger})_{ij}R_{jk} = R_{ij}(R^{\dagger})_{jk} = \delta_{ik}$ (orthogonality / unitarity).

In total, the Hamiltonian **operator** may not change, which is compensated by rotation of the creators and annihiltors, so
$$
H = \sum_{i, j} c^{\dagger}_i H_{ij} c_j = \sum_{i, j, k, l, m, n} c^{\dagger}_m R_{mi}(R^{\dagger})_{ik} H_{kl} R_{lj} (R^{\dagger})_{jn} c_n .
$$

We derive the transformation of the U matrix elements by applying the same rotation to all occurring creators and annihilators and demanding that the total quartic part of the Hamiltonian operator remains the same, so
$$
H = \frac{1}{2} c^{\dagger}_m R_{mi} c^{\dagger}_n R_{nj} (R^{\dagger})_{iw} (R^{\dagger})_{jx} U_{wxyz} R_{yk} R_{zl} (R^{\dagger})_{lo} c_o (R^{\dagger})_{kp} c_p = \frac{1}{2} c^{\dagger}_i c^{\dagger}_j U_{ijkl} c_l c_k.
$$
and therefore the rotation is applied to the U matrix as
$$
U'_{ijkl} = (R^{\dagger})_{iw} (R^{\dagger})_{jx} U_{wxyz} R_{yk} R_{zl}.
$$

### Fabrizio's phonons

In [5]:
import numpy as np
import os,sys

In [6]:
lbd=1
alpha22=1
NBANDS=4

In [8]:
alpha22=alpha22**2
interactionmatrix=np.zeros([NBANDS,2,NBANDS,2,NBANDS,2,NBANDS,2])
list_of_indices=np.array([[1,2,4,3],\
                          [2,1,3,4],\
                          [1,3,4,2],\
                          [3,1,2,4],\
                          [2,3,3,2],\
                          [3,2,2,3],\
                          [1,4,4,1],\
                          [4,1,1,4],\
                          [2,4,3,1],\
                          [4,2,1,3],\
                          [3,4,2,1],\
                          [4,3,1,2]])
x_coeffs=-1*np.array([-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1])
y_coeffs=-1*np.array([+1,+1,-1,-1,-1,-1,-1,-1,-1,-1,+1,+1])

for iband in range(4):
    for jband in range(4):
        for kband in range(4):
            for lband in range(4):
                orbvector=[iband+1,jband+1,kband+1,lband+1]
                for itest in range(np.shape(list_of_indices)[0]):

                    if np.array_equal(orbvector,list_of_indices[itest]):
                        interactionmatrix[iband,0,jband,0,kband,0,lband,0] = -1*lbd*alpha22*(x_coeffs[itest]+(y_coeffs[itest]))
                        interactionmatrix[iband,1,jband,1,kband,1,lband,1] = -1*lbd*alpha22*(x_coeffs[itest]+(y_coeffs[itest]))
                        break
                

In [9]:
kanamori_umatrix = np.genfromtxt("u_matrix_template.dat",dtype='str')
np.shape(kanamori_umatrix)
kanamori_umatrix[0:3,:]

for iline in range(np.shape(kanamori_umatrix)[0]):
    uvalue=float(kanamori_umatrix[iline,8])
    iband=int(kanamori_umatrix[iline,0])-1
    jband=int(kanamori_umatrix[iline,2])-1
    kband=int(kanamori_umatrix[iline,4])-1
    lband=int(kanamori_umatrix[iline,6])-1
    if kanamori_umatrix[iline,1]=="u":
        ispin=0
    else:
        ispin=1
    if kanamori_umatrix[iline,3]=="u":
        jspin=0
    else:
        jspin=1
    if kanamori_umatrix[iline,5]=="u":
        kspin=0
    else:
        kspin=1
    if kanamori_umatrix[iline,7]=="u":
        lspin=0
    else:
        lspin=1
    interactionmatrix[iband,ispin,jband,jspin,kband,kspin,lband,lspin] += uvalue

In [10]:
def sud(ispin):
    if(ispin)==0:
        spinstring="u"
    else:
        spinstring="d"
    return spinstring


fname="u_matrix.dat"
f=open(fname,'w')
f.write(' '.join([str(NBANDS),"BANDS"]))
f.write("\n")
for iband in range(4):
    for ispin in range(2):
        for jband in range(4):
            for jspin in range(2):
                for kband in range(4):
                    for kspin in range(2):
                        for lband in range(4):
                            for lspin in range(2):
                                uvalue=interactionmatrix[iband,ispin,jband,jspin,kband,kspin,lband,lspin]
                                if abs(uvalue) > 1e-15:
                                    orbvector=[str(iband+1),sud(ispin),str(jband+1),sud(jspin),str(kband+1),sud(kspin),str(lband+1),sud(lspin)]
                                    for icomp in range(8):
                                        f.write(orbvector[icomp])
                                        f.write(" ")
                                    f.write(str(uvalue))
                                    f.write("\n")
f.close()